In [5]:
%matplotlib widget
from IPython.display import HTML
from components import MainWindow

with open("./src/styling.css") as f:
    style = f.read()
    
HTML(style)

window = MainWindow()
window.box